<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84(%2B%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8A%B8)_RAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [182]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [183]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

In [184]:
# 첫 설치 이후 설치 안함
!pip install fredapi

In [185]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv
from fredapi import Fred

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
# start_date = today - timedelta(days=375)

# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 전략명
str_strategy = 'RAA '

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} {str_strategy}.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 전략에 사용할 ETF 명 리스트에 담기
etf_stock = ['QQQ', 'IWN']
etf_bond = ['IEF', 'TLT']
etf_real = ['GLD']
etf_canary = ['VWO', 'BND']

# 자산군 리스트 더하기, 중복 제거
etf_order = list(set(etf_stock + etf_bond + etf_real + etf_canary))

# ETF 가격 추출
df_etf = yf.download(etf_order, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0).dropna()
display(df_etf)

# 1개월씩 자르기
df_etf_monthly = df_etf[::MBD]

[*********************100%%**********************]  7 of 7 completed


,QQQ,IEF,BND,GLD,VWO,TLT,IWN
2023-09-15,370.8100,93.0900,70.7900,178.3400,40.5400,92.9600,141.0000
2023-09-14,377.2700,93.3800,70.9300,177.1700,40.7500,93.5300,142.3800
2023-09-13,374.2100,93.5900,71.0500,177.0800,40.5100,94.2100,139.9800
2023-09-12,372.7900,93.4800,70.9400,177.4900,40.5500,94.2300,140.9500
2023-09-11,376.9700,93.3500,70.8900,178.4100,40.6600,93.6900,140.7500
...,...,...,...,...,...,...,...
2007-04-16,39.2966,56.2565,45.5330,68.4000,27.2092,54.2871,60.5503
2007-04-13,38.9390,56.1951,45.4904,67.8400,26.8727,53.9897,59.8035
2007-04-12,38.8605,56.2906,45.5633,66.9900,26.7853,54.1570,59.4265
2007-04-11,38.5553,56.2360,45.5694,67.0800,26.4488,54.1446,59.0495


In [186]:
## DataFrame 만들기: 미국 실업률 (08-01: 8월1일~31일 실업률 통계)
fred = Fred(api_key='5958f7e2cfc6adda8b4efaa3c131d32e')
fred_unrate = fred.get_series('UNRATE').sort_index(ascending=False)
df_fred_unrate = pd.DataFrame(fred_unrate, columns=['unemployment rate'])

str_fred_unrate = str_today + str_strategy + "### 미국 실업률"
display(Markdown(str_fred_unrate))
display(df_fred_unrate[:5])

with open(output_path + filename, 'w') as f:
  f.write(f'{str_fred_unrate}\n')
  df_fred_unrate[:5].to_csv(f, index=True)

### 2023-09-18 RAA ### 미국 실업률

,unemployment rate
2023-08-01,3.8000
2023-07-01,3.5000
2023-06-01,3.6000
2023-05-01,3.7000
2023-04-01,3.4000


In [187]:
# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-09-18 RAA ### ETF 현재-1-3-6-12 개월 가격

,QQQ,IEF,BND,GLD,VWO,TLT,IWN
today/2023-09-15,370.8100,93.0900,70.7900,178.3400,40.5400,92.9600,141.0000
1mo_ago/2023-08-16,362.5400,93.2015,70.5966,175.5700,39.9400,93.5594,142.6300
3mo_ago/2023-06-15,369.7528,96.5639,72.3137,181.9100,41.8622,102.1331,141.6200
6mo_ago/2023-03-16,305.9161,96.7131,72.0703,178.5700,38.7012,103.5827,135.4203
12mo_ago/2022-09-14,293.9344,96.3561,71.3453,157.9300,39.1056,104.7604,140.3099


In [188]:
## DataFrame 만들기: 모멘텀 스코어
df_etf_monthly_copy = df_etf_monthly.copy(deep=True)
df_etf_momentum_score = pd.DataFrame(index=df_etf_monthly.index, columns=df_etf_monthly.columns)

for i in range(0, len(df_etf_monthly) - 12):
  ror_1mo = df_etf_monthly_copy.pct_change(-1).iloc[0]  # 1개월 수익률
  ror_3mo = df_etf_monthly_copy.pct_change(-3).iloc[0]  # 3개월 수익률
  ror_6mo = df_etf_monthly_copy.pct_change(-6).iloc[0]  # 6개월 수익률
  ror_12mo = df_etf_monthly_copy.pct_change(-12).iloc[0]  # 12개월 수익률
  df_etf_monthly_copy.drop(df_etf_monthly_copy.index[0], inplace=True)

  # 가중치 더하기
  df_etf_momentum_score.iloc[i] = ((12 * ror_1mo) + (4 * ror_3mo) + (2 * ror_6mo) + (ror_12mo)) * 100

df_etf_momentum_score.dropna(inplace=True)

str_etf_momentum_score = str_today + str_strategy + "### ETF 별 1-3-6-12 개월 Momentum Score"
display(Markdown(str_etf_momentum_score))
display(df_etf_momentum_score)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_momentum_score}\n')
  df_etf_momentum_score[:5].to_csv(f, index=True)

### 2023-09-18 RAA ### ETF 별 1-3-6-12 개월 Momentum Score

,QQQ,IEF,BND,GLD,VWO,TLT,IWN
2023-09-15,97.0971,-26.7074,-9.4724,23.7483,18.5638,-75.3885,-6.7326
2023-08-16,18.3852,-70.1248,-47.0119,-63.4759,-56.5348,-155.9381,-19.9750
2023-07-18,244.9615,-11.6110,-0.8586,28.2868,11.5612,-31.5605,98.2680
2023-06-15,334.3426,-16.2569,0.6959,12.0859,111.4666,-9.4366,136.2389
2023-05-16,95.8964,29.8388,15.9266,58.3526,-28.3006,-4.8639,-147.9537
...,...,...,...,...,...,...,...
2008-08-08,63.8227,12.9710,-4.8951,-115.8951,-124.4829,-0.1517,149.2700
2008-07-10,-99.5065,47.6713,27.8610,178.6600,-165.5256,64.4206,-190.4772
2008-06-10,51.7459,-18.9098,-17.9902,-19.2635,-75.2151,-18.6406,12.1886
2008-05-09,98.7568,9.7852,22.2163,-27.5658,94.9432,-10.0211,-0.5294


In [189]:
## DataFrame 만들기: ETF 종목, 종목별 투자 비율
row_count = len(df_etf_momentum_score)
df_high_etf = pd.DataFrame(columns=[0, 1, 2, 3, 4], index=df_etf_momentum_score.index)

df_buy_ratio_etf = pd.DataFrame(columns=[0, 1, 2, 3, 4], index=df_etf_momentum_score.index)
bond_portfolio_ratio = [0.5] *2 + [0] * 3
stock_portfolio_ratio = [0.2] * 5

# 마지막에 출력할 str 정의
str_fred_unrate = ''
str_canary_minus = ''
str_buy_etf = ''
str_etf_ratio = ''

for i in reversed(range(row_count)):
  # 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%, 채권 40%, 실물자산 20%)
  if df_fred_unrate.iloc[i][0] > df_fred_unrate.iloc[i+12][0]:
    str_fred_unrate = 'True'

  # 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?
    if df_etf_momentum_score.iloc[i]['VWO'] < 0 or df_etf_momentum_score.iloc[i]['BND'] < 0:
      str_canary_minus = 'True'

      # 하나라도 마이너스이면 채권 포트폴리오 투자 (IEF 50%, TLT 50%)
      df_high_etf.iloc[i] = [etf_bond[0]] + [etf_bond[1]] * 4
      df_buy_ratio_etf.iloc[i] = bond_portfolio_ratio

    # 둘 다 플러스이면 5개 자산 비율 투자
    else:
      df_high_etf.iloc[i] = etf_stock + etf_bond + etf_real
      df_buy_ratio_etf.iloc[i] = stock_portfolio_ratio

      str_canary_minus = 'False'

  # 지금 실업률이 1년전 실업률보다 낮음
  else:
    df_high_etf.iloc[i] = etf_stock + etf_bond + etf_real
    df_buy_ratio_etf.iloc[i] = stock_portfolio_ratio

    str_fred_unrate = 'False'
    str_canary_minus = ''

  # 최신 데이터로 종목 뽑기
  if i == 0:
    str_buy_etf = ' '.join(df_high_etf.iloc[i].drop_duplicates())
    str_etf_ratio = str(round(df_buy_ratio_etf[df_buy_ratio_etf > 0].iloc[i].drop_duplicates().dropna()[0] * 100)) + ' %'

str_high_etf = str_today + str_strategy + "### ETF 투자 종목 선정"
display(Markdown(str_high_etf))
display(df_high_etf)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_high_etf}\n')
  df_high_etf[:5].to_csv(f, index=True)

### 2023-09-18 RAA ### ETF 투자 종목 선정

,0,1,2,3,4
2023-09-15,IEF,TLT,TLT,TLT,TLT
2023-08-16,QQQ,IWN,IEF,TLT,GLD
2023-07-18,QQQ,IWN,IEF,TLT,GLD
2023-06-15,QQQ,IWN,IEF,TLT,GLD
2023-05-16,QQQ,IWN,IEF,TLT,GLD
...,...,...,...,...,...
2008-08-08,IEF,TLT,TLT,TLT,TLT
2008-07-10,IEF,TLT,TLT,TLT,TLT
2008-06-10,IEF,TLT,TLT,TLT,TLT
2008-05-09,QQQ,IWN,IEF,TLT,GLD


In [190]:
# 종목별 투자 비율
df_buy_ratio = df_buy_ratio_etf

str_buy_ratio = str_today + str_strategy + "### ETF 투자 비율"
display(Markdown(str_buy_ratio))
display(df_buy_ratio)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_buy_ratio}\n')
  df_buy_ratio[:5].to_csv(f, index=True)

### 2023-09-18 RAA ### ETF 투자 비율

,0,1,2,3,4
2023-09-15,0.5000,0.5000,0,0,0
2023-08-16,0.2000,0.2000,0.2000,0.2000,0.2000
2023-07-18,0.2000,0.2000,0.2000,0.2000,0.2000
2023-06-15,0.2000,0.2000,0.2000,0.2000,0.2000
2023-05-16,0.2000,0.2000,0.2000,0.2000,0.2000
...,...,...,...,...,...
2008-08-08,0.5000,0.5000,0,0,0
2008-07-10,0.5000,0.5000,0,0,0
2008-06-10,0.5000,0.5000,0,0,0
2008-05-09,0.2000,0.2000,0.2000,0.2000,0.2000


In [191]:
## 백테스트
monthly_column_count = df_high_etf.shape[1]
monthly_index = df_high_etf.index

# 비율대로 돈 투자
df_invest_money = pd.DataFrame(columns=[i for i in range(monthly_column_count)], index=monthly_index)
df_invest_count = pd.DataFrame(columns=[i for i in range(monthly_column_count)], index=monthly_index)
df_cash = pd.DataFrame(columns=['cash'], index=monthly_index)

# 초기자본
invest_cash = 10_000

for i in reversed(range(row_count)):
  row_sum = 0

  for j in range(monthly_column_count):
    if i == row_count - 1:
      row_sum = invest_cash
      break
    else:
      sel_etf = df_high_etf.iloc[i+1][j]
      row_sum += df_invest_count.iloc[i+1][j] * df_etf_monthly.iloc[i][sel_etf]

  for j in range(monthly_column_count):
    df_cash.iloc[i][0] = row_sum
    sel_etf = df_high_etf.iloc[i][j]
    sel_etf_price = df_etf_monthly.iloc[i][sel_etf]
    sel_etf_buy_count = df_cash.iloc[i][0] * df_buy_ratio.iloc[i][j] / sel_etf_price
    df_invest_count.iloc[i][j] = sel_etf_buy_count

display(Markdown(str_today + str_strategy + '### ETF 구매 개수'))
display(df_invest_count)
display(Markdown(str_today + str_strategy + '### 평가 금액'))
display(df_cash)

CAGR = str(round(((df_cash.iloc[0][0] / df_cash.iloc[-1][0]) ** (12/(row_count-1)) - 1)*100, 2))
display(Markdown(f'{str_today[1:]} {str_strategy} ## CAGR: {CAGR} %'))

### 2023-09-18 RAA ### ETF 구매 개수

,0,1,2,3,4
2023-09-15,190.0848,190.3506,0.0000,0.0000,0.0000
2023-08-16,19.4473,49.4315,75.6471,75.3577,40.1573
2023-07-18,19.2285,50.1704,77.0147,73.2804,40.3899
2023-06-15,19.7243,51.4979,75.5265,71.4081,40.0920
2023-05-16,21.4610,54.2370,71.5762,68.9139,37.9269
...,...,...,...,...,...
2008-08-08,78.1474,82.2883,0.0000,0.0000,0.0000
2008-07-10,78.5872,81.8251,0.0000,0.0000,0.0000
2008-06-10,78.0417,82.3932,0.0000,0.0000,0.0000
2008-05-09,46.5464,38.9856,30.8382,32.5527,22.4560


### 2023-09-18 RAA ### 평가 금액

,cash
2023-09-15,35389.9797
2023-08-16,35252.0898
2023-07-18,37085.9612
2023-06-15,36465.6596
2023-05-16,35057.7711
...,...
2008-08-08,9978.9761
2008-07-10,10047.0026
2008-06-10,9722.3949
2008-05-09,9815.5192


## 2023-09-18  RAA  ## CAGR: 8.54 %

In [192]:
## 결과 출력
display(Markdown(str_today[1:] + str_strategy))

str_result_etf = '## 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%  채권 40%  실물자산 20%): ' + str_fred_unrate
display(Markdown(str_result_etf))

str_result_momentum = '## 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?: ' + str_canary_minus
display(Markdown(str_result_momentum))

str_result_buy = '## 구매할 자산: ' + str_buy_etf
display(Markdown(str_result_buy))

str_result_ratio = '## 각각 구매 비율: ' + str_etf_ratio
display(Markdown(str_result_ratio))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_result_etf}\n')
  f.write(f'{str_result_momentum}\n')
  f.write(f'{str_result_buy}\n')
  f.write(f'{str_result_ratio}\n')

## 2023-09-18 RAA 

## 지금 실업률이 1년전 실업률보다 높은가? (낮으면 5개 자산 투자. 주식 40%  채권 40%  실물자산 20%): True

## 높으면 VWO, BND 1-3-6-12 모멘텀스코어가 하나라도 마이너스인가?: True

## 구매할 자산: IEF TLT

## 각각 구매 비율: 50 %